# Memory

https://python.langchain.com/api_reference/langchain/memory.html

챗봇으로 하여금 대화(상태)를 '기억'하게끔 한다

Memory maintains Chain state, incorporating context from past runs.


In [1]:
# Langchain 의 memory 계층
#  BaseMemory --> BaseChatMemory --> <name>Memory  # Examples: ZepMemory, MotorheadMemory

In [2]:
# OpenAI 사에서 제공하는 '기본 API' 도 랭체인 없이 사용 가능.
# 메모리 지원하지 않는다.  이전 대화 기억 못함.  stateless 하다!

# ChatGPT 서비스 는 '메모리' 기능이 탑재되어 있다.
# 챗봇이 이전의 대화 내용과 질문을 기억하고 답할수 있다.

# 기본 import

In [3]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.prompts.chat import ChatPromptTemplate

# ConversationBufferMemory

In [4]:
# ConversationBufferMemory
# 대화 내용 '전체'를 저장하는 메모리

# 장점: 단순하다

# 단점:
# => 매번 요청할때마다 '이전 대화 기록 전체' 를 같이 보내야 함.
#  그래야 모델이 전에 일어났던 대화를 보고 이해 할수 있다.
#  대화내용이 길어질수록 메모리도 계속 커지니까 성능적으로도 & 비용적으로도 비효율적이다.

In [5]:
# v0.3
from langchain.memory.buffer import ConversationBufferMemory
# https://python.langchain.com/api_reference/langchain/memory/langchain.memory.buffer.ConversationBufferMemory.html#conversationbuffermemory



In [6]:
memory = ConversationBufferMemory()

memory.save_context(
    {"input": "Hi!"},
    {'output': 'How are you?'}
)

memory.load_memory_variables({})  # history buffer 리턴

C:\Users\User\AppData\Local\Temp\ipykernel_20160\346152885.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


{'history': 'Human: Hi!\nAI: How are you?'}

## return_messages=True
history 에  AIMessage 와 HumanMessage 로 저장된다.

In [7]:
memory = ConversationBufferMemory(return_messages=True)

memory.save_context(
    {"input": "Hi!"},
    {'output': 'How are you?'}
)

memory.load_memory_variables({}) 

{'history': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='How are you?', additional_kwargs={}, response_metadata={})]}

In [8]:
memory.save_context(
    {"input": "Hi!"},
    {'output': 'How are you?'}
)

memory.load_memory_variables({}) 

{'history': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='How are you?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='How are you?', additional_kwargs={}, response_metadata={})]}

# ConversationBufferWindowMemory

In [9]:
# v0.3
from langchain.memory.buffer_window import ConversationBufferWindowMemory
# https://python.langchain.com/api_reference/langchain/memory/langchain.memory.buffer_window.ConversationBufferWindowMemory.html#conversationbufferwindowmemory


In [10]:
# ConversationBufferWindowMemory 는 대화의 '특정 부분만' 을 저장하는 메모리.

# 장점:
#   메모리를 특정 크기로 유지할 수 있다!
#   따라서 모든 대화 내용을 저장하지 않아도 된다!

# 단점:
#   챗봇이 전체 대화가 아닌 '최근 대화' 에만 집중하게 된다.

In [11]:
memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,   # 버퍼 윈도우 사이즈,  몇개의 메세지를 저장할지 지정.
)

C:\Users\User\AppData\Local\Temp\ipykernel_20160\2681781434.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


In [12]:
# 도우미 함수 하나 준비.  여러 메세지를 추가하기 편리하니까
def add_message(input, output):
  memory.save_context({"input": input}, {"output": output})

In [13]:
add_message("1", "1")
add_message("2", "2")
add_message("3", "3")
add_message("4", "4")

In [14]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1', additional_kwargs={}, response_metadata={}),
  AIMessage(content='1', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='2', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='3', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='4', additional_kwargs={}, response_metadata={}),
  AIMessage(content='4', additional_kwargs={}, response_metadata={})]}

In [15]:
add_message("5", "5")
memory.load_memory_variables({})

{'history': [HumanMessage(content='2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='2', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='3', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='4', additional_kwargs={}, response_metadata={}),
  AIMessage(content='4', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='5', additional_kwargs={}, response_metadata={}),
  AIMessage(content='5', additional_kwargs={}, response_metadata={})]}

# ConversationSummaryMemory

In [16]:
# ConversationSummaryMemory 는 LLM 을 사용하여 대화의 요약본 (summary) 생성

In [17]:
llm = ChatOpenAI(temperature=0.1)

In [18]:
# v0.3
from langchain.memory.summary import ConversationSummaryMemory
# https://python.langchain.com/api_reference/langchain/memory/langchain.memory.summary.ConversationSummaryMemory.html#langchain.memory.summary.ConversationSummaryMemory

# Continually summarizes the conversation history.
# The summary is updated after each conversation turn.
# The implementations returns a summary of the conversation history
# which can be used to provide context to the model.



In [19]:
memory = ConversationSummaryMemory(llm = llm)

C:\Users\User\AppData\Local\Temp\ipykernel_20160\3817587095.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm = llm)


In [20]:
# 함수 하나 만들어 두고..

def get_history():
  return memory.load_memory_variables({})

In [21]:
# message 추가
add_message(
    "Hi I'm John, I live in South Korea",    # input
    "Wow that is so cool!"  # output : AI 답변
    )

In [22]:
# 또 message 추가
add_message(
    "South Korea is so pretty",
    "I wish I could go!!!")

In [23]:
get_history()

{'history': 'John from South Korea introduces himself to the AI, who responds enthusiastically, finding it cool. John mentions how pretty South Korea is, and the AI expresses a wish to visit.'}

In [24]:
# ↑ 대화를 '요약' 한 내용으로 기억하고 있다
# 대화의 turn 이 길어질수로 summary 가 각 메세지를 효율적으료 '요약(압축)' 해준다

# ConversationSummaryBufferMemory

In [25]:
# v0.3
from langchain.memory.summary_buffer import ConversationSummaryBufferMemory
# https://python.langchain.com/api_reference/langchain/memory/langchain.memory.summary_buffer.ConversationSummaryBufferMemory.html#langchain.memory.summary_buffer.ConversationSummaryBufferMemory

# Buffer with summarizer for storing conversation memory.
# Provides a running summary of the conversation together with
#  the most recent messages in the conversation under the constraint
#   that the total number of tokens in the conversation does not exceed a certain limit.


In [26]:
# ConversationSummaryBufferMemory 는
#   ConversationBufferMemory 와 ConversationSummaryMemory 의 결합형

# 메모리에 보내온 '메세지의 수'를 지정하여 저장한다.
# 오래된 메세지들 또한 '요약' 하여 저장함.

In [27]:
llm = ChatOpenAI(temperature=0.1)

In [28]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,   # 최대 가용한 메세지 토큰수 (메세지 요약되기 전)
    return_messages=True,
)

C:\Users\User\AppData\Local\Temp\ipykernel_20160\3924822954.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


In [29]:
add_message(
    "Hi I'm John, I live in South Korea",    # input
    "Wow that is so cool!"  # output : AI 답변
    )
get_history()

{'history': [HumanMessage(content="Hi I'm John, I live in South Korea", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={})]}

In [30]:
add_message(
    "South Korea is so pretty",
    "I wish I could go!!!")
get_history()

{'history': [HumanMessage(content="Hi I'm John, I live in South Korea", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='South Korea is so pretty', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I wish I could go!!!', additional_kwargs={}, response_metadata={})]}

In [31]:
add_message(
    "How far is Korea from Argentina?",
    "I don't know! Super far!"
)
get_history()

{'history': [HumanMessage(content="Hi I'm John, I live in South Korea", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='South Korea is so pretty', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I wish I could go!!!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Korea from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kwargs={}, response_metadata={})]}

In [32]:
# 아래 셀을 여러차레 해보자  약 (3,4번?)
# =>실제 '요약'이 발생할때면 Model IO 가 발생하기 때문에 시간이 좀 걸리는게 느껴질거다!

add_message(
    "How far is Brazil from Argentina?",
    "I don't know! Super far!"
)
get_history()

{'history': [HumanMessage(content="Hi I'm John, I live in South Korea", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='South Korea is so pretty', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I wish I could go!!!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Korea from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Brazil from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kwargs={}, response_metadata={})]}

In [33]:
# limit 에 도달하면, 오래된 메세지들이 요약되고 있을 것을 확인할수 있다. (SystemMessage 확인)

# ★그러나 '요약' 이라는 과정은 API 를 사용한다는 사실을 명심하세요.
# ★'요약' 동작은 비용 지출이 발생되는 부분입니다.

# ConversationKGMemory
Knowledge Graph Memory

In [34]:
# v0.3
from langchain_community.memory.kg import ConversationKGMemory
# https://python.langchain.com/api_reference/community/memory/langchain_community.memory.kg.ConversationKGMemory.html

# Knowledge graph conversation memory.
# Integrates with external knowledge graph to store and retrieve information about knowledge triples in the conversation.

In [35]:
# 대화중에 '엔티티'의 knowledge graph 를 형성한다 => 가장 중요한 것들만 추출한 요약본.
# knowledge graph 는 history 를 가지고 오지 않는다.  대신 '엔티티' 를 가지고 옴


In [36]:
memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)

In [37]:
add_message(
    "Hi I'm John, I live in South Korea",    # input
    "Wow that is so cool!"  # output : AI 답변
    )

In [38]:
memory.load_memory_variables({'input': 'who is John'})

{'history': [SystemMessage(content='On John: John lives in South Korea.', additional_kwargs={}, response_metadata={})]}

In [39]:
# 여기에 메세지를 더해보자
add_message("John likes kimchi", "Wow that is so cool!")

In [40]:
memory.load_memory_variables({'input': 'What does John like'})

{'history': [SystemMessage(content='On John: John lives in South Korea. John likes kimchi.', additional_kwargs={}, response_metadata={})]}

# Memory on Chain

In [ ]:
# 메모리를 'chain' 에 꽂는 방법.
# 두가지 형태의 'chain' 에 각각 꽂는 방법.
#    1. off-the-shelf chain  : LangChain 에서 '일반적인 목적'을 수행하는 기본 제공되는 chain
#    2. LCEL 사용한 chain : 커스텀 chain

## LLMChain

In [41]:
# off-the-shelf chain 중 하나
# v0.3
from langchain.chains.llm import LLMChain

In [42]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
)

chain = LLMChain(
    llm=llm,
    memory=memory, # ★메모리를 chain 에 꽂기!
    prompt=PromptTemplate.from_template("{question}")
)

C:\Users\User\AppData\Local\Temp\ipykernel_20160\4183264304.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [43]:
chain.invoke(input={'question':"My name is John"})

{'question': 'My name is John',
 'history': '',
 'text': 'Nice to meet you, John! How can I assist you today?'}

In [ ]:
"""
{'question': 'My name is John',
 'history': '',    <- 최초에는 history 없다!
 'text': 'Nice to meet you, John! How can I assist you today?'}
"""
None

In [44]:
chain.invoke(input={'question':"I live in Seoul"})

{'question': 'I live in Seoul',
 'history': 'Human: My name is John\nAI: Nice to meet you, John! How can I assist you today?',
 'text': "That's great! Seoul is a vibrant and bustling city with a rich history and culture. There are so many things to see and do in Seoul, from exploring ancient palaces and temples to enjoying delicious Korean cuisine and shopping in trendy neighborhoods. What do you enjoy most about living in Seoul?"}

In [ ]:
"""
{'question': 'I live in Seoul',
    ↓ 이전 대화에 대한 history 가 생겼다 <- memory 가 동작한다는 뜻!
 'history': 'Human: My name is John\nAI: Nice to meet you, John! How can I assist you today?',
 'text': "That's great! Seoul is a vibrant and bustling city with a rich history and culture. There are so many things to see and do in Seoul, from exploring ancient palaces and temples to enjoying delicious Korean cuisine and shopping in trendy neighborhoods. What do you enjoy most about living in Seoul?"}
"""
None

In [45]:
chain.invoke(input={'question':"What is my name?"})

{'question': 'What is my name?',
 'history': "System: The human introduces himself as John. The AI responds by greeting John and asking how it can assist him today.\nHuman: I live in Seoul\nAI: That's great! Seoul is a vibrant and bustling city with a rich history and culture. There are so many things to see and do in Seoul, from exploring ancient palaces and temples to enjoying delicious Korean cuisine and shopping in trendy neighborhoods. What do you enjoy most about living in Seoul?",
 'text': "I'm sorry, I do not know your name as I am an AI assistant and do not have access to personal information."}

In [ ]:
"""
{'question': 'What is my name?',
 'history': "System: The human introduces himself as John. The AI responds by greeting John and asking how it can assist him today.\nHuman: I live in Seoul\nAI: That's great! Seoul is a vibrant and bustling city with a rich history and culture. There are so many things to see and do in Seoul, from exploring ancient palaces and temples to enjoying delicious Korean cuisine and shopping in trendy neighborhoods. What do you enjoy most about living in Seoul?",
 'text': "I'm sorry, I do not know your name as I am an AI assistant and do not have access to personal information."}

↑ 어라? 모른다고?

chain 을 디버깅 해보자!
"""
None

## verbose=

In [46]:
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
    verbose=True,  # chain 을 실행했을때 chain 의 프롬프트 로그들을 확인할수 있다. (디버깅용)
)

chain.invoke(input={'question':"My name is John"})
chain.invoke(input={'question':"I live in Seoul"})
chain.invoke(input={'question':"What is my name?"})



> Entering new LLMChain chain...
Prompt after formatting:
My name is John

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
I live in Seoul

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
What is my name?

> Finished chain.


{'question': 'What is my name?',
 'history': "System: John introduces himself and mentions that he lives in Seoul. The AI responds by acknowledging Seoul's vibrant and bustling nature, rich history, and culture. It asks John what he enjoys most about living in Seoul and then learns John's name. The AI greets John by name and offers assistance.\nAI: , the capital city of South Korea. It is a vibrant and bustling metropolis with a rich history and culture. There are so many things to see and do in Seoul, from exploring ancient palaces and temples to shopping in trendy neighborhoods and enjoying delicious Korean cuisine. I love living in this dynamic city and experiencing all that it has to offer.",
 'text': "I'm sorry, I do not have access to personal information such as your name."}

In [ ]:
# ↑ chain 의 prompt 로그 들을 확인할 수 있다.
# 보다시피 대화의 내역(history) 가 prompt에 계속 추가되진 않는다.

# 우리가 원하는 어떤 방식으로 prompt에게 대화 기록(history)을 추가해줘야 한다!


In [49]:
# memory 에는 저장이 되어 있다
print(memory.load_memory_variables({})['history'])

System: John introduces himself and mentions that he lives in Seoul. The AI responds by acknowledging Seoul's vibrant and bustling nature, rich history, and culture. It asks John what he enjoys most about living in Seoul and then learns John's name. The AI greets John by name and offers assistance, expressing its love for living in the dynamic city of Seoul and experiencing all that it has to offer.
Human: What is my name?
AI: I'm sorry, I do not have access to personal information such as your name.


In [ ]:
# 이제 Memory 클래스 에게
# 템플릿 앗에 콘텐츠를 넣으라고 예기 해주어야 한다.
# ↓ 이는 memory_key= 에 "chat_history"(예시) 라고 말해주기만 하면 된다.

## memory_key=

In [52]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    memory_key="chat_history",  # 이렇게만 지정해 놓아도 prompt template 에서 삽입 가능!
)

template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory, # ★메모리를 chain 에 꽂기!
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

In [53]:
chain.invoke(input={'question':"My name is John"})



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human:My name is John
    You:


> Finished chain.


{'question': 'My name is John',
 'chat_history': '',
 'text': 'Nice to meet you, John! How can I assist you today?'}

In [54]:
chain.invoke(input={'question':"I live in Seoul"})



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is John
AI: Nice to meet you, John! How can I assist you today?
    Human:I live in Seoul
    You:


> Finished chain.


{'question': 'I live in Seoul',
 'chat_history': 'Human: My name is John\nAI: Nice to meet you, John! How can I assist you today?',
 'text': "That's great to hear! How can I assist you with information or tasks related to Seoul?"}

In [55]:
chain.invoke(input={'question':"What is my name?"})



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is John
AI: Nice to meet you, John! How can I assist you today?
Human: I live in Seoul
AI: That's great to hear! How can I assist you with information or tasks related to Seoul?
    Human:What is my name?
    You:


> Finished chain.


{'question': 'What is my name?',
 'chat_history': "Human: My name is John\nAI: Nice to meet you, John! How can I assist you today?\nHuman: I live in Seoul\nAI: That's great to hear! How can I assist you with information or tasks related to Seoul?",
 'text': 'Your name is John.'}

In [56]:
chain.invoke(input={'question':"My name is John"})
chain.invoke(input={'question':"I live in Seoul"})



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is John
AI: Nice to meet you, John! How can I assist you today?
Human: I live in Seoul
AI: That's great to hear! How can I assist you with information or tasks related to Seoul?
Human: What is my name?
AI: Your name is John.
    Human:My name is John
    You:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    System: The human introduces himself as John. The AI responds by greeting John and asking how it can assist him today.
Human: I live in Seoul
AI: That's great to hear! How can I assist you with information or tasks related to Seoul?
Human: What is my name?
AI: Your name is John.
Human: My name is John
AI: Nice to meet you, John! How can I assist you today?
    Human:I live in Seoul
    You:


> Finished chain.


{'question': 'I live in Seoul',
 'chat_history': "System: The human introduces himself as John. The AI responds by greeting John and asking how it can assist him today.\nHuman: I live in Seoul\nAI: That's great to hear! How can I assist you with information or tasks related to Seoul?\nHuman: What is my name?\nAI: Your name is John.\nHuman: My name is John\nAI: Nice to meet you, John! How can I assist you today?",
 'text': "That's great to hear! How can I assist you with information or tasks related to Seoul?"}

In [ ]:
# ↑ 지금까지는 텍스트 기반의 history  였다
#  ChatModel 에서 Message기반의 대화기록은 어떻게 적용할 것인가?

# Chat Based Memory

In [57]:
memory.load_memory_variables({})

{'chat_history': "System: John introduces himself and mentions he lives in Seoul. The AI responds by greeting John and offering assistance with information or tasks related to Seoul.\nHuman: What is my name?\nAI: Your name is John.\nHuman: My name is John\nAI: Nice to meet you, John! How can I assist you today?\nHuman: I live in Seoul\nAI: That's great to hear! How can I assist you with information or tasks related to Seoul?"}

## return_messages=True

In [67]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",  
    return_messages=True,   # history 를 '문자열' 이 아닌 "Message"로 바꾸어 리턴함
)

In [ ]:
# 과연 prompt 에는 어떻게 대화 history 들을 넘겨줄수 있을까?
#  단순한 하나의 텍스트가 아니라... Human Message - AI Message - Human Messagbe - AI Message - .... (여러 메세지들)
#  심지여 요약본 발생시 System message 도 있을텐데?

# prompt 에 이를 위한 공간을 어케 만드나?
#  => MessagePlaceHolder

## MessagePlaceHolder

In [68]:
# v0.3
from langchain_core.prompts.chat import MessagesPlaceholder

# https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.MessagesPlaceholder.html#langchain_core.prompts.chat.MessagesPlaceholder

# Prompt template that assumes variable is already list of messages.
# A placeholder which can be used to pass in a list of messages.

In [69]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),

    # ↓ 누가 보냈는지 알수 없는, (AI? Human? System?)
    #  예측하기 어려운 메세지들. 양과 제한 없는 메세지를 가질수 있다.
    MessagesPlaceholder(variable_name='chat_history'),
    
    ('human', "{question}"),
])

In [70]:
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt, # 위에서 작성한 프롬프트!
    verbose=True,
)

In [71]:
chain.invoke(input={'question':"My name is John"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is John

> Finished chain.


{'question': 'My name is John',
 'chat_history': [HumanMessage(content='My name is John', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello John! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'text': 'Hello John! How can I assist you today?'}

In [72]:
chain.invoke(input={'question':"I live in Seoul"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is John
AI: Hello John! How can I assist you today?
Human: I live in Seoul

> Finished chain.


{'question': 'I live in Seoul',
 'chat_history': [HumanMessage(content='My name is John', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello John! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I live in Seoul', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?', additional_kwargs={}, response_metadata={})],
 'text': 'Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?'}

In [73]:
chain.invoke(input={'question':"What is my name?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is John
AI: Hello John! How can I assist you today?
Human: I live in Seoul
AI: Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?
Human: What is my name?

> Finished chain.


{'question': 'What is my name?',
 'chat_history': [HumanMessage(content='My name is John', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello John! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I live in Seoul', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is John.', additional_kwargs={}, response_metadata={})],
 'text': 'Your name is John.'}

# LCEL Based Memory

In [74]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [79]:
chain = prompt | llm

chain.invoke({
    'chat_history': memory.load_memory_variables({})['chat_history'],
    'question': 'My name is John',
})

AIMessage(content='Hello John! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 24, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BrMv1Ww0BTV6a9lt5oVKfs5kMiI9i', 'finish_reason': 'stop', 'logprobs': None}, id='run--ba17bbfa-7995-47a2-a4a4-daff272acfb6-0', usage_metadata={'input_tokens': 24, 'output_tokens': 10, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
"""
↑ 위 방법도 가능은 하다...
단, 위 접근 방식의 문제는
우리가 chain 을 호출할 때마다 chat_history 도 추가해줘야 한다는 거다.

↓ 이보다 더 좋은 방법도 있다. 바로 'Runnables' 라는 것을 사용하는 것이다
"""
None

## RunnablePassThrough

In [80]:
# v0.3
from langchain_core.runnables.passthrough import RunnablePassthrough
# https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html#langchain_core.runnables.passthrough.RunnablePassthrough

# Runnable to passthrough inputs unchanged or with additional keys.
# This Runnable behaves almost like the identity function,
#  except that it can be configured to add additional keys to the output,
#  if the input is a dict.

In [81]:
# 메모리 변수 리턴하는 함수
def load_memory():
    return memory.load_memory_variables({})['chat_history']

chain = RunnablePassthrough.assign(chat_history=load_memory)| prompt | llm

chain.invoke({
    'question': 'My name is John',
})

# chain 호출할때 chat_history 를 매번 기입할 필요 없어졌다.!

TypeError: load_memory() takes 0 positional arguments but 1 was given

In [82]:
def load_memory(input):
    print('🎃 load_memory()', input)  # 확인용
    return memory.load_memory_variables({})['chat_history']

chain = RunnablePassthrough.assign(chat_history=load_memory)| prompt | llm

chain.invoke({
    'question': 'My name is John',
})

🎃 load_memory() {'question': 'My name is John'}


AIMessage(content='Hello John! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 24, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BrN42FfePVWEriYT9nQseh3HW849x', 'finish_reason': 'stop', 'logprobs': None}, id='run--22ce2035-d3c5-434c-88fc-56b168259ad5-0', usage_metadata={'input_tokens': 24, 'output_tokens': 10, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
# 또 한가지 문제
# chain 호출후 결과를 다시 메모리에 추가해야 함.

# 좋은 방법은 chain 호출 함수를 따로 만들어 보자.

In [83]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def load_memory(input):
  print("🎃load_memory()", input)
  return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})

    # 체인 호출 결과를 메모리에 저장
    memory.save_context(
        {'input': question},
        {'output': result.content},
    )

    print(result)

In [84]:
invoke_chain('My name is John')

🎃load_memory() {'question': 'My name is John'}
content='Hello John! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 24, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BrN8p6FHQTZ7HJJQBaVLlv1WRXksg', 'finish_reason': 'stop', 'logprobs': None} id='run--231388a1-d877-44b2-b0e8-29815a942a2b-0' usage_metadata={'input_tokens': 24, 'output_tokens': 10, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [85]:
invoke_chain("What is my name?")

🎃load_memory() {'question': 'What is my name?'}
content='Your name is John.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 47, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BrN9NsHXws4QzWQzkZjtfxZ3fDJ73', 'finish_reason': 'stop', 'logprobs': None} id='run--4537796c-e0ef-4d06-b5bd-7d5a7e1e943f-0' usage_metadata={'input_tokens': 47, 'output_tokens': 5, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [86]:
get_history()

{'chat_history': [HumanMessage(content='My name is John', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello John! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is John.', additional_kwargs={}, response_metadata={})]}

## memory_key= 삭제

In [87]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    # memory_key="chat_history",   # memory_key="history" 
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),  # 변경!
        ("human", "{question}"),
    ]
)

def load_memory(input):
  print("🎃load_memory()", input)
  return memory.load_memory_variables({})["history"]  # 변경!

                                # ↓ 변경
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})

    # 체인 호출 결과를 메모리에 저장
    memory.save_context(
        {'input': question},
        {'output': result.content},
    )

    print(result)

invoke_chain('My name is John')    

🎃load_memory() {'question': 'My name is John'}
content='Hello John! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 24, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BrNEqV1nbIejba4kvtePMMvtCpT69', 'finish_reason': 'stop', 'logprobs': None} id='run--61812f7b-7e43-445e-ba58-11133e23f265-0' usage_metadata={'input_tokens': 24, 'output_tokens': 10, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [88]:
invoke_chain("What is my name?")

🎃load_memory() {'question': 'What is my name?'}
content='Your name is John.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 47, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BrNF9vgaoSsNZbKViYv1DbFu0gRnf', 'finish_reason': 'stop', 'logprobs': None} id='run--d74db093-700d-4654-b74c-145cd052374a-0' usage_metadata={'input_tokens': 47, 'output_tokens': 5, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
